In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    horizontal_flip=True
)

base_model = VGG16(weights="imagenet", include_top=False, input_shape=(28, 28, 1))

for layer in base_model.layers[-5:]:
    layer.trainable = True

model = tf.keras.Sequential([
    tf.keras.layers.Resizing(32, 32),  
    tf.keras.layers.Conv2D(3, (3, 3), padding="same", input_shape=(28, 28, 1)),  
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation="relu"),
    BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation="relu"),
    BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation="softmax")
])

model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    metrics=[tf.keras.metrics.CategoricalAccuracy()]
)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)

history = model.fit(
    datagen.flow(train_images.reshape(-1, 28, 28, 1), train_labels, batch_size=64),
    epochs=1,
    validation_data=(test_images.reshape(-1, 28, 28, 1), test_labels),
    callbacks=[early_stopping]
)

test_loss, test_acc = model.evaluate(test_images.reshape(-1, 28, 28, 1), test_labels, verbose=2)
print(f"Точность на тестовых данных: {test_acc}")

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
train_images.shape, train_labels.shape, test_images.shape, test_labels.shape

In [ ]:
model.summary()

In [ ]:
# Після навчання можна порівняти точність цієї моделі з багатошаровою нейронною мережею, 
# створеною на попередньому етапі. Очікується,
# що використання VGG16, як глибшої мережі, дасть вищу точність завдяки її здатності витягувати складніші ознаки.